<a href="https://colab.research.google.com/github/SergeyHSE/LinearRegressor.github.io/blob/main/RegressionAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Analysis of descriptive statistics
2. Оценивание регрессионной модели в рамках предпосылок классической линейной регрессионной модели (КЛРМ)
3. Оценивание регрессионной модели в условиях отклонений от предпосылок КЛРМ:
 - Diagnosis of single atypical observations
 - Diagnosis of sample homogeneity
 - Diagnostics of specification errors and model correction when they are detected
 - Diagnosis of multicollinearity and model correction when it is detected
 - Diagnosis of heteroscedasticity and model correction when it is detected
 - Diagnosis of endogeneity and model correction when it is detected


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.diagnostic as dg
import scipy.stats
from scipy.stats import boxcox

When studying the literature on this topic, the most interesting articles were "Economic efficiency of beef cattle production in Thailand" by Professor Suneeporn Suwanmaneepong of King Mongkut's Institute of Technology Ladkrabang Faculty of Agricultural Technology PhD and "Assessment of technical efficiency and its determinants in beef cattle production in Kenya" by Eric Ruto of Lincoln University. In this paper, the professor describes the economic efficiency of livestock production. To build the model she uses the following variables as the most efficient ones: cost of feed and additives, equipment, drugs and labor, access to priority markets, etc. Unfortunately, our data do not contain information on the costs of purchasing veterinary drugs, so we will not be able to analyze their impact on the profitability of the enterprise. Therefore, we will do something else: we will deduct from the cost price all the cost items that we have. This will give us the amount including the costs of veterinary drugs.

Moreover, both authors conclude in the conclusions of their studies that there is a need for government intervention with different types of assistance such as:
- Improving farmers' access to the knowledge they need to develop their farms as well as their farming skills
- Providing access to more modern technologies
- Improving access to market services
- Creating opportunities for off-farm income generation.

All these factors are in one way or another related to government support, to a certain type of subsidy, which directly, according to the authors, should improve profit margins, and therefore improve the model's performance.


In [2]:
from google.colab import files
file = files.upload()

Saving agro_census.dta to agro_census.dta


In [16]:
data = pd.read_stata('agro_census.dta')
data.columns, data.shape

(Index(['NPPP', 'COD_COATO', 'KFS', 'KOPF', 'OKVED', 'land_total',
        'cost_milk_KRS', 'cost_KRS_food', 'cost_meat_KRS', 'AB_1', 'CF_1',
        'short_credit', 'long_credit', 'debit_debt', 'credit_debt',
        'gov_sup_plant', 'gov_sup_seed', 'gov_sup_grain', 'subs_plant',
        'subs_grain', 'gov_sup_farming', 'gov_sup_KRS', 'subs_prod_farm',
        'subs_milk', 'subs_meat', 'subs_KRS', 'subs_combikorm', 'sub_chemistry',
        'subs_fuel', 'farms_number', 'profit_farms_number',
        'unprofit_farms_number', 'capital', 'profit', 'unprofit', 'J', 'O',
        'empl_org', 'empl_prod', 'V', 'W', 'X', 'AN', 'AO', 'AP', 'AQ', 'AR',
        'BE', 'BF', 'BG', 'BQ', 'BR', 'BS', 'BT', 'BU', 'BY', 'BZ', 'CA',
        'salary_plant', 'salary_farm', 'DB', 'DC', 'DF', 'DG', 'DH', 'DI', 'DK',
        'DO', 'DT', 'EC', 'EG', 'EJ', 'EK', 'ER', 'ES', '_merge'],
       dtype='object'),
 (6287, 76))

In [17]:
df = data[(data['OKVED'] == '01.21')]
df.shape

(2595, 76)

In [19]:
df['net_profit'] = df['profit'] - df['unprofit']
df['other_cost'] = df['BQ'] - df['salary_farm'] - df['DC'] - df['DI']
df['subsidies'] = df['gov_sup_KRS'] + df['subs_prod_farm'] + df['subs_milk'] + df['subs_KRS'] + df['subs_combikorm'] + df['subs_fuel']
df['debt'] = df['credit_debt'] - df['debit_debt']
df['cost_services'] = df['J'] - df['BQ']

<ipython-input-19-2521fcf124cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['net_profit'] = df['profit'] - df['unprofit']
<ipython-input-19-2521fcf124cc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['other_cost'] = df['BQ'] - df['salary_farm'] - df['DC'] - df['DI']
<ipython-input-19-2521fcf124cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

In [20]:
df.rename(columns={'DC' : 'amortization',
                   'DI' : 'social_cost'}, inplace=True)
df['output'] = df['AP'] + df['BE'] + df['BS']

<ipython-input-20-d2f9976e03fa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'DC' : 'amortization',
<ipython-input-20-d2f9976e03fa>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['output'] = df['AP'] + df['BE'] + df['BS']


In [21]:
column_names = ['net_profit', 'other_cost', 'subsidies', 'debt',
                'cost_services', 'amortization', 'output', 'salary_farm',
                'empl_org', 'KOPF', 'social_cost']
livestock = df[column_names]
livestock.shape

(2595, 11)

We ended up with the following variables:
net_profit - net profit of livestock production
rentabel - profitability (net profit to revenue ratio)
other_cost - costs, which include, among other things, costs for repayment of loans and for purchase of veterinary drugs.
social_cost - deductions for social needs
subsidies - total amount of subsidies, including subsidies for milk and meat production, fuel subsidies, etc.
debt - current short-term debts (difference between accounts payable and accounts receivable).
cost_services - costs of realization of services, works (difference between the cost of sold goods, products, works, services and the cost of sale of livestock products
amortization - amortization
output - gross output of milk, meat, cattle.
salary_farm - labor costs.
empl_org - Average annual number of employees of the agricultural organization
KOPF - (
42
Unitary enterprises, based on the right of economic management
47
Open joint stock companies
52
Production cooperatives
65
Limited liability companies
67
Closed joint-stock companies
54
Collective farms
55
State farms)

In [22]:
livestock.head()

,net_profit,other_cost,subsidies,debt,cost_services,amortization,output,salary_farm,empl_org,KOPF,social_cost
0,8931.0,30701.0,3892.0,26477.0,48314.0,2114.0,2693.0,8735.0,294,47,1147
1,2495.0,13939.0,3710.0,11271.0,3138.0,582.0,1678.0,3827.0,166,52,490
2,98.0,16900.0,4096.0,18952.0,7885.0,572.0,3043.0,6790.0,235,67,1014
3,-4868.0,6460.0,207.0,10718.0,2083.0,162.0,668.0,1432.0,95,52,182
6,-3457.0,7502.0,825.0,3060.0,8783.0,0.0,332.0,1645.0,136,67,210


In [23]:
# Calculate NaN
livestock.isnull().sum()

net_profit       0
other_cost       0
subsidies        0
debt             0
cost_services    0
amortization     0
output           0
salary_farm      0
empl_org         0
KOPF             0
social_cost      0
dtype: int64

In [25]:
# Calculate zeros
(livestock == 0).sum()

net_profit        25
other_cost        70
subsidies        249
debt              25
cost_services     78
amortization     154
output           101
salary_farm       82
empl_org           0
KOPF               0
social_cost       90
dtype: int64

In [31]:
livestock = livestock.loc[~(livestock == 0).all(axis=1)]
livestock.shape

(2595, 11)

In [32]:
livestock.describe()

,net_profit,other_cost,subsidies,debt,cost_services,amortization,output,salary_farm,empl_org,KOPF,social_cost
count,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000,2595.000000
mean,7841.578343,22817.386898,4313.216817,14084.024663,10753.023815,1647.998073,2848.882312,7750.120617,152.261657,55.442389,1179.604239
std,26857.090812,35745.317554,8416.003095,49316.495885,24358.144435,2661.216427,7633.231854,9493.783825,129.003540,8.535656,1579.778162
min,-117594.000000,-26608.000000,0.000000,-642913.000000,-72855.000000,0.000000,0.000000,0.000000,1.000000,42.000000,0.000000
25%,134.500000,6547.000000,510.500000,916.000000,1391.000000,224.000000,629.000000,2532.000000,81.000000,52.000000,323.000000
50%,2889.000000,13543.000000,1669.000000,5085.000000,4462.000000,742.000000,1292.000000,5323.000000,125.000000,52.000000,710.000000
75%,9903.000000,26494.000000,4685.000000,15042.000000,10921.500000,1900.000000,2368.000000,9783.000000,189.500000,65.000000,1476.000000
max,732684.000000,771320.000000,163506.000000,842063.000000,529553.000000,32719.000000,120690.000000,225988.000000,2251.000000,67.000000,27666.000000
